In [ ]:
# default_exp ads

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
!pip install -qU nbdev fastcore

In [ ]:
#hide
!pip install -qU gspread facebook_business python-facebook-api

In [ ]:
#hide
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


In [ ]:
#hide
#not deps but we need them to use nbdev and run tests
from nbdev import * 
from nbdev.showdoc import *
from fastcore.test import *

## Facebook Auth

In [ ]:
#hide
%%capture

import yaml
with open(path.parent/"facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env AD_ACC_ID {data.get("AD_ACC_ID")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
from tomorrowswig_followers.core import *
from tomorrowswig_followers.followers import get_new_followers

In [ ]:
#export
import os
from typing import *

from datetime import datetime, timedelta
from functools import partial

import pandas as pd
import numpy as np
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.ad import Ad
from facebook_business.adobjects.adaccount import AdAccount

AD_ACC_ID = os.environ.get("AD_ACC_ID")

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Ads reporting

In [ ]:
#export
FacebookAdsApi.init(APP_ID, APP_SECRET, TOKEN)
account = AdAccount(AD_ACC_ID)

In [ ]:
datetime.fromtimestamp(1595946718)

datetime.datetime(2020, 7, 28, 14, 31, 58)

In [ ]:
datetime(2020,7,20,0,0).strftime('%s')

'1595203200'

In [ ]:
insights = account.get_insights(fields=[
    "ad_id",
    "ad_name",
    "clicks",
    "cpc",
    "ctr",
    "cpm",
    "cost_per_action_type",
    "spend",
    "impressions",
    "reach",
    "actions",
    "video_avg_time_watched_actions",
    "video_p50_watched_actions",
    "video_p75_watched_actions",
    "video_p95_watched_actions",

], params={
    'level': "ad",
    'date_preset': "yesterday",
    # "date_start": "1595116800",
    # "date_stop": "1595203200"
})


In [ ]:
from facebook_business.adobjects.adsinsights import *
fields = [f for f in AdsInsights.Field.__dict__ if not f.startswith("__")]

In [ ]:
" ".join([fields[56], fields[105]])

'cost_per_unique_conversion unique_conversions'

In [ ]:
account.get_insights(fields=["ad_id", "quality_ranking", "engagement_rate_ranking",
                             "conversion_rate_ranking", "ad_name"], params={
    'level': AdsInsights.Level.ad,
    'date_preset': AdsInsights.DatePreset.yesterday,
})

[<AdsInsights> {
    "ad_id": "23844703941280691",
    "ad_name": "3 - Indonesia - General Ad 2-D",
    "conversion_rate_ranking": "UNKNOWN",
    "date_start": "2020-08-04",
    "date_stop": "2020-08-04",
    "engagement_rate_ranking": "UNKNOWN",
    "quality_ranking": "UNKNOWN"
}, <AdsInsights> {
    "ad_id": "23844751184140691",
    "ad_name": "2 - Mexico - General Ad #2-D",
    "conversion_rate_ranking": "UNKNOWN",
    "date_start": "2020-08-04",
    "date_stop": "2020-08-04",
    "engagement_rate_ranking": "UNKNOWN",
    "quality_ranking": "UNKNOWN"
}, <AdsInsights> {
    "ad_id": "23845206704740691",
    "ad_name": "3 - Philippines",
    "conversion_rate_ranking": "UNKNOWN",
    "date_start": "2020-08-04",
    "date_stop": "2020-08-04",
    "engagement_rate_ranking": "UNKNOWN",
    "quality_ranking": "UNKNOWN"
}]

In [ ]:
# account.get_insights(fields=fields[:56] + fields[57:105] + fields[106:], params={
#     'level': AdsInsights.Level.ad,
#     'date_preset': AdsInsights.DatePreset.last_3d,
# })[2]

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df

In [ ]:
#export
COLUMNS = ["Ad Name", "Ads Followers Increase", "% (Clicks)", "% (Impressions)", "Cost Per Follow",
           "Clicks (All)", "Link Clicks", "CPC (All)" , "CPC (Cost per Link Click)",
           "CTR (All)", "CPM (Cost per 1,000 Impressions)", "Amount Spent (USD)", "Impressions", "Reach", "Post Reactions", "Post Shares",
           "Video Average Play Time", "Video Plays at 50%", "Video Plays at 75%", "Video Plays at 95%"]

In [ ]:
#export
get_insights = partial(account.get_insights, fields=[
        "ad_id",
        "ad_name",
        "clicks",
        "cpc",
        "ctr",
        "cpm",
        "cost_per_action_type",
        "spend",
        "impressions",
        "reach",
        "actions",
        "video_avg_time_watched_actions",
        "video_p50_watched_actions",
        "video_p75_watched_actions",
        "video_p95_watched_actions",
    ], params={
        'level': "ad",
        'date_preset': "yesterday",
    })

In [ ]:
#export
def get_action(cell, name):
    action = [a for a in cell if a["action_type"] == name]
    if action:
        return float(action[0]["value"])
    return 0

In [ ]:
#export
def get_followers_change(date: datetime) -> pd.DataFrame:
    history_df = get_df("History")
    new_followers = get_dif(history_df)
    followers_date = (date + timedelta(days=1)).strftime("%b %d %Y")
    new_followers = new_followers.iloc[:,new_followers.columns.str.startswith(followers_date)].replace(0, np.nan)
    new_followers = new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)
    if not new_followers.values.size:
        new_followers["no change"] = 0
    return new_followers

In [ ]:
assert get_followers_change(datetime.strptime("2020-07-25", "%Y-%m-%d")).columns[0] == "Jul 26 2020                 2020-07-26 19:10:34"
assert get_followers_change(datetime.strptime("2031-07-25", "%Y-%m-%d")).sum().item() == 0

In [ ]:
#export
def add_country_total(df: pd.DataFrame) -> Tuple[pd.DataFrame, str]:
    dupes = df[df.index.duplicated(False)]
    if not len(dupes):
        return

    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
    total = dupes.iloc[0,:].copy()
    index = dupes.index[0]
    total["ad_name"] = f"{index} total"
    total[to_mean] = dupes[to_mean].apply(np.mean)
    total[to_sum] = dupes[to_sum].astype(int).apply(sum)
    return df.append(total), index

In [ ]:
#export
def add_followers(df: pd.DataFrame, date: datetime):
    new_followers = get_followers_change(date)
    df["Ads Followers Increase"] = new_followers
    df.fillna(0, inplace=True)
    df["% (Clicks)"] = (df["Ads Followers Increase"] / df["clicks"]* 100).astype(int)
    df["% (Impressions)"] = (df["Ads Followers Increase"] / df["impressions"] * 100)
    df["Cost Per Follow"] = (df["spend"] / df["Ads Followers Increase"])
    df.replace([np.inf], 0, inplace=True)

In [ ]:
ad = Ad("23844695610150691")

In [ ]:
ad = ad.api_get(fields=["targeting"])
ad["targeting"]["geo_locations"]["countries"][0]

'CA'

In [ ]:
#export
def get_countries(ids: List[str]) -> List[Dict]:
    countries = []
    for id in ids:
        ad = Ad(id).api_get(fields=["targeting"])
        country = ad["targeting"]["geo_locations"]["countries"][0]
        countries.append({"ad_id": id, "country": country})
    return countries

In [ ]:
assert get_countries(["23844695610150691", "23844698190420691"]) == [{'ad_id': '23844695610150691', 'country': 'CA'},
                                                                     {'ad_id': '23844698190420691', 'country': 'US'}]

In [ ]:
def create_first_part():
    df = pd.DataFrame(get_insights())
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(pd.DataFrame(get_countries(df["ad_id"].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)

    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    add_followers(df, date)
    return df.round(2), date

In [ ]:
df, date = create_first_part()

In [ ]:
to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time",
            "% (Clicks)", "% (Impressions)", "Cost Per Follow"]
to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
total = pd.Series(name="total", index=df.columns, dtype=float)

In [ ]:
total[to_mean] = df[to_mean].apply(np.mean)
total[to_sum] = df[to_sum].apply(sum)

In [ ]:
df.append(total).round(2)

,ad_name,clicks,cpc,cpm,ctr,impressions,reach,spend,Post Reactions,Post Shares,Link Clicks,CPC (Cost per Link Click),Video Average Play Time,Video Plays at 50%,Video Plays at 75%,Video Plays at 95%,Ads Followers Increase,% (Clicks),% (Impressions),Cost Per Follow
country,,,,,,,,,,,,,,,,,,,,
MX,2 - Mexico - General Ad #2-D,82.0,0.06,0.41,0.70,11757.0,11757.0,4.79,5.0,3.0,47.0,0.10,2.0,1514.0,457.0,265.0,0.0,0.0,0.0,0.0
ID,3 - Indonesia - General Ad 2-D,263.0,0.03,0.27,1.01,25984.0,25984.0,6.90,7.0,6.0,116.0,0.06,2.0,4020.0,1219.0,747.0,0.0,0.0,0.0,0.0
PH,3 - Philippines,92.0,0.02,0.36,1.87,4908.0,4895.0,1.76,0.0,0.0,47.0,0.04,2.0,865.0,317.0,192.0,0.0,0.0,0.0,0.0
total,NaN,437.0,0.04,0.35,1.19,42649.0,42636.0,13.45,12.0,9.0,210.0,0.07,2.0,6399.0,1993.0,1204.0,0.0,0.0,0.0,0.0


In [ ]:
#export
def add_total(df: pd.DataFrame) -> pd.DataFrame:
    to_mean = ["cpc", "cpm", "ctr", "CPC (Cost per Link Click)", "Video Average Play Time",
                "% (Clicks)", "% (Impressions)", "Cost Per Follow"]
    to_sum = set(df.columns).difference(set(to_mean + ["ad_name"]))
    total = pd.Series(name="total", index=df.columns, dtype=float)
    total[to_mean] = df[to_mean].apply(np.mean)
    total[to_sum] = df[to_sum].apply(sum)
    return df.append(total)

In [ ]:
#export
def get_insights_df(insights: List) -> Tuple[pd.DataFrame, datetime]:
    df = pd.DataFrame(insights)
    floats = ["cpc", "cpm", "ctr", "spend"]
    df[floats] = df[floats].astype(float)
    ints = ["clicks", "impressions", "reach"]
    df[ints] = df[ints].astype(int)

    df = df.merge(pd.DataFrame(get_countries(df["ad_id"].values)))
    df.set_index(df["country"], inplace=True)
    df.sort_values("ad_name", inplace=True)
    date = datetime.strptime(df["date_start"].values[0], "%Y-%m-%d")

    df["Post Reactions"] = df["actions"].apply(partial(get_action, name="post_reaction"))
    df["Post Shares"] = df["actions"].apply(partial(get_action, name="post"))
    df["Link Clicks"] = df["actions"].apply(partial(get_action, name="link_click"))
    df["CPC (Cost per Link Click)"] = df["cost_per_action_type"].apply(partial(get_action, name="link_click"))
    get_video_action = partial(get_action, name="video_view")
    df["Video Average Play Time"] = df["video_avg_time_watched_actions"].apply(get_video_action)
    df["Video Plays at 50%"] = df["video_p50_watched_actions"].apply(get_video_action)
    df["Video Plays at 75%"] = df["video_p75_watched_actions"].apply(get_video_action)
    df["Video Plays at 95%"] = df["video_p95_watched_actions"].apply(get_video_action)

    df.drop(columns=["actions", "cost_per_action_type", "date_start", "date_stop",
                     "video_avg_time_watched_actions", "ad_id", "country"] +
            [f"video_p{p}_watched_actions" for p in [50,75,95]], inplace=True)
    
    dupes = add_country_total(df)
    add_followers(df, date)
    df = add_total(df)
    df.rename(columns={"ad_name": "Ad Name", "clicks": "Clicks (All)", "cpc": "CPC (All)",
                       "ctr": "CTR (All)", "cpm": "CPM (Cost per 1,000 Impressions)",
                       "reach": "Reach", "impressions": "Impressions", "spend": "Amount Spent (USD)"}, inplace=True)
    return df[COLUMNS].round(2), date

In [ ]:
insights = get_insights()
df, date = get_insights_df(insights)

In [ ]:
stats = pd.Series(dtype=float, name="followers")
ads_followers_gain = df.loc["total", "Ads Followers Increase"]
followers_gain = get_followers_change(date).sum().item()
spent = df.loc["total", "Amount Spent (USD)"]

In [ ]:
stats["Ads Followers Increase"] = ads_followers_gain
stats["New Followers"] = get_new_followers()[1]
stats["Random Followers"] = stats["New Followers"] - ads_followers_gain
stats["Unfollowers"] = stats["New Followers"] - followers_gain
stats["Real Ads Followers Increase"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

In [ ]:
stats_df = pd.DataFrame(stats)
stats_df.insert(0, "  ", stats_df.index)

In [ ]:
stats_df["Conversion Rate"] = stats_df["followers"] / df.loc["total", "Clicks (All)"]
stats_df["Cost Per Follower/Increase"] = spent/ stats_df["followers"]
stats_df.iloc[2:5, 2] = 0
stats_df.iloc[2:4, 3] = 0

random followers + followers from ads - unfollowers = followers gain

unfollowers = random followers + followers from ads - followers gain = new followers - followers gain

random_followers = new followers - followers from ads

In [ ]:
#export
def more_stats(df: pd.DataFrame, date: datetime) -> pd.DataFrame:
    stats = pd.Series(dtype=float, name="followers")
    ads_followers_gain = df.loc["total", "Ads Followers Increase"]
    followers_gain = get_followers_change(date).sum().item()
    spent = df.loc["total", "Amount Spent (USD)"]
    stats["Ads Followers Increase"] = ads_followers_gain
    stats["New Followers"] = get_new_followers()[1]
    stats["Random Followers"] = stats["New Followers"] - ads_followers_gain
    stats["Unfollowers"] = stats["New Followers"] - followers_gain
    stats["Real Ads Followers Increase"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

    stats_df = pd.DataFrame(stats)
    stats_df.index.name = "  "
    # stats_df.insert(0, "  ", stats_df.index)
    stats_df["Conversion Rate"] = stats_df["followers"] / df.loc["total", "Clicks (All)"]
    stats_df[" "] = 0
    stats_df["Cost Per Follower/Increase"] = spent/ stats_df["followers"]
    stats_df.iloc[2:5, 1] = 0
    stats_df.iloc[2:4, 3] = 0
    return stats_df.round(2)

In [ ]:
#export
def create_insights(event: Dict = None, context=None,) -> str:
    insights = get_insights()
    df, date = get_insights_df(insights)
    worksheet_name = date.strftime("%b %d %Y")
    stats_df = more_stats(df, date)
    empty = pd.Series(name="", dtype=str)
    df = df.append([empty, empty, empty])
    write_df(df, worksheet_name)
    write_df(stats_df, worksheet_name, loc=f"B{len(df)}")

    wsh = get_worksheet(worksheet_name)
    wsh.format(f"C1:G{len(df)-1}", {"textFormat": {
        "foregroundColor": {
            "red": 0.45,
            "green": 0.0,
            "blue": 0.0
        }}},)
    wsh.format(f"D:E", {"numberFormat": {"type": "PERCENT"}},)
    wsh.format(f"F", {"numberFormat": {"type": "CURRENCY"}},)

    unf_cell = wsh.find("Unfollowers")
    wsh.format(f"C{wsh.find('Unfollowers').row}", {"textFormat": {
        "foregroundColor": {
            "red": 0.0,
            "green": 0.0,
            "blue": 1.0
        }}},)
    return f"Created insights in '{worksheet_name}'"

In [ ]:
create_insights()

"Created insights in 'Aug 04 2020'"

In [ ]:
# DONE estimate, real
## Fix unfollowers 30 min, 70 min
## relevance score 60 min, 30 min
## Match country:
# Get country from API 1 hour, 2.5 hours

# SHARE
# https://www.facebook.com/business/help/403110480493160?id=561906377587030
## check all fields data

# TODO
## Formatting 120 min, 120 min


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
